In [ ]:
!pip install --ignore-installed pycaret[full]

In [1]:
# import libraries
import boto3, os
from sagemaker import get_execution_role
import pandas as pd
from dotenv import load_dotenv
from load_data import load_data



my_region = boto3.session.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


In [2]:
load_dotenv(".env")
role = get_execution_role()

target = os.getenv("target")

bucket= os.getenv("bucket")
data_key = os.getenv("key")
data_location = 's3://{}/{}'.format(bucket, data_key)
print(data_location)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
s3://streaming-data-platform-ml-data/bakerloo.csv


In [3]:
data = load_data(data_location)
df = data.copy()
df.shape

(7127, 5)

In [4]:
print(df.shape)
# Randomly shuffle the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)
# Sort by day and then pick the first 80% as your test data. 
train_size = int(0.8 * len(df))
train_data = df_shuffled[:train_size]
test_data = df_shuffled[train_size:]
target = os.getenv("target")
print(train_data.shape, test_data.shape)

(7127, 5)
(5701, 5) (1426, 5)


In [5]:
from pycaret.regression import *
s = setup(data = train_data, target = target, session_id=123)

,Description,Value
0,Session id,123
1,Target,Bakerloo10
2,Target type,Regression
3,Original data shape,"(5701, 5)"
4,Transformed data shape,"(5701, 5)"
5,Transformed train set shape,"(3990, 5)"
6,Transformed test set shape,"(1711, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


In [6]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7094,1.1429,1.0631,0.9796,0.0660,0.0487,0.1200
rf,Random Forest Regressor,0.6516,1.1838,1.0786,0.9790,0.0624,0.0389,0.0490
xgboost,Extreme Gradient Boosting,0.7022,1.1904,1.0871,0.9787,0.0682,0.0470,0.0120
lightgbm,Light Gradient Boosting Machine,0.7363,1.2312,1.1031,0.9781,0.0719,0.0511,0.3630
et,Extra Trees Regressor,0.6243,1.2674,1.1152,0.9774,0.0625,0.0358,0.0380
gbr,Gradient Boosting Regressor,0.8901,1.7100,1.3008,0.9695,0.0916,0.0682,0.0220
dt,Decision Tree Regressor,0.6944,1.7523,1.3153,0.9686,0.0726,0.0383,0.0040
knn,K Neighbors Regressor,0.9090,1.7993,1.3355,0.9679,0.0875,0.0616,0.0070
lar,Least Angle Regression,1.1603,2.6602,1.6245,0.9523,0.1327,0.0966,0.0040
lr,Linear Regression,1.1603,2.6602,1.6245,0.9523,0.1327,0.0966,0.2780


In [7]:
# Users are allowed to create their own model if they want to.
# This uses the default hyperparameters to train the model which you can then choose to tune as you want.
# In order to tune hyperparameters, the tune_model function is used
xgboost = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7425,1.3530,1.1632,0.9722,0.0703,0.0481
1,0.7465,1.6436,1.2820,0.9760,0.0768,0.0548
2,0.6960,0.9967,0.9983,0.9794,0.0579,0.0432
3,0.7119,1.2304,1.1092,0.9784,0.0740,0.0528
4,0.7201,1.2051,1.0978,0.9786,0.0749,0.0540
5,0.7338,1.1022,1.0499,0.9792,0.0751,0.0444
6,0.6379,0.8248,0.9082,0.9842,0.0512,0.0388
7,0.7117,1.2044,1.0975,0.9810,0.0678,0.0450
8,0.6910,1.1891,1.0905,0.9798,0.0702,0.0475


In [8]:
tuned_model = tune_model(xgboost)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7354,1.1626,1.0782,0.9761,0.0617,0.0441
1,0.7243,1.6121,1.2697,0.9764,0.0748,0.0476
2,0.6604,0.8831,0.9397,0.9818,0.0567,0.0392
3,0.7427,1.3866,1.1775,0.9757,0.0714,0.0488
4,0.7017,1.1311,1.0635,0.9799,0.0815,0.0549
5,0.7052,0.9118,0.9549,0.9828,0.0710,0.0441
6,0.6298,0.7056,0.8400,0.9865,0.0525,0.0384
7,0.6837,1.0436,1.0216,0.9835,0.0611,0.0413
8,0.6851,1.1648,1.0792,0.9802,0.0639,0.0417


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [9]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [10]:
predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.6944,1.2472,1.1168,0.9782,0.0715,0.0494


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
2676,24,3,10,10,23.0,23.793894
724,17,3,5,29,14.0,14.018951
4595,26,0,18,17,28.0,27.176376
1112,18,5,10,40,18.0,17.882286
4064,22,1,13,44,23.0,22.350794
...,...,...,...,...,...,...
3854,24,1,20,25,22.0,23.127800
4481,30,3,7,14,29.0,29.343656
467,31,3,7,42,29.0,29.768587
3551,2,6,4,37,2.0,1.878369


In [11]:
final_best_model = finalize_model(best)
print(final_best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                             'minute'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x29f8ca8f0>)])


In [12]:
final_prediction = predict_model(final_best_model, data=test_data)
final_prediction.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.7110,1.4438,1.2016,0.9757,0.0731,0.0497


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
5701,19,5,10,2,19.0,18.888796
5702,28,3,17,26,29.0,28.140224
5703,27,0,8,39,27.0,26.623402
5704,29,3,16,48,29.0,28.634963
5705,21,1,10,8,23.0,22.864239


In [13]:
from pycaret.utils.generic import check_metric
check_metric(final_prediction.Bakerloo10, final_prediction.prediction_label, 'RMSE')

1.2016